In [1]:
'''
Mulliken population analysis with IAO orbitals
'''

import numpy
from functools import reduce
from sys import stdout
from pyscf import gto, scf, lo
from pyscf.tools.dump_mat import dump_rec
from pyscf.tools.dump_mat import dump_mo

mol = gto.M(atom="""
  C1    0.0000000    0.0000000   -0.6654946
  H1    0.0000000    0.9374306   -1.2121600
  H1   -0.0000000   -0.9374306   -1.2121600
  C2   -0.0000000    0.0000000    0.6654946
  H2    0.0000000    0.9374306    1.2121600
  H2   -0.0000000   -0.9374306    1.2121600
""", basis='cc-pvdz')

mol.build()

nao_h2o = mol.nao // 2

h2o_1_idx = gto.search_ao_label(mol, ["C1", "H1"])
h2o_2_idx = gto.search_ao_label(mol, ["C2", "H2"])

print(h2o_1_idx)
print(h2o_2_idx)

mf = scf.RHF(mol).run()
c_ao_lo = lo.orth_ao(mf, 'nao')

assert numpy.linalg.norm(reduce(numpy.dot, [c_ao_lo.T, mf.get_ovlp(), c_ao_lo]) - numpy.eye(c_ao_lo.shape[0])) < 1e-8

[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23]
[24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47]
converged SCF energy = -78.0396286884589


In [2]:
# check the localality
from scipy.linalg import sqrtm
ww = numpy.einsum("mn,np->mp", sqrtm(mf.get_ovlp()), c_ao_lo)
w2 = ww**2

for i in range(nao_h2o):    
    assert abs(1.0 - sum(w2[h2o_1_idx, i])) < 2e-1
    assert abs(sum(w2[h2o_2_idx, i])) < 2e-1
    assert abs(1.0 - sum(w2[h2o_2_idx, i + nao_h2o])) < 2e-1
    assert abs(sum(w2[h2o_1_idx, i + nao_h2o])) < 2e-1

In [3]:
# transform mo_occ to IAO representation. Note the AO dimension is reduced
ovlp_ao = mf.get_ovlp()
dm_ao = mf.make_rdm1()
dm_lo = reduce(numpy.dot, [c_ao_lo.T, ovlp_ao, dm_ao, ovlp_ao, c_ao_lo])
assert numpy.linalg.norm(reduce(numpy.dot, [dm_lo/2, dm_lo/2]) - dm_lo/2) < 1e-8

idx_a = h2o_1_idx
idx_b = h2o_2_idx

dm_aa = dm_lo[numpy.ix_(idx_a, idx_a)]
dm_bb = dm_lo[numpy.ix_(idx_b, idx_b)]
dm_ab = dm_lo[numpy.ix_(idx_a, idx_b)]

In [4]:
from scipy import linalg

uab, sab, vhab = linalg.svd(dm_ab)
numpy.linalg.norm(
    dm_ab - reduce(numpy.dot, [uab, numpy.diag(sab), vhab])
)

1.0271198377762134e-15

In [5]:
from pyscf import tools

ua, sa, vha = linalg.svd(dm_aa)
numpy.linalg.norm(
    dm_aa - reduce(numpy.dot, [ua, numpy.diag(sa), vha])
)

tol = 1e-2
cor_idx_a = []
act_idx_a = []
ext_idx_a = []

for p, sp in enumerate(sa):
    if abs(sp - 2.0) < tol:
        cor_idx_a.append(p)
    elif abs(sp) < tol:
        ext_idx_a.append(p)
    else:
        act_idx_a.append(p)

ub, sb, vhb = linalg.svd(dm_bb)
numpy.linalg.norm(
    dm_bb - reduce(numpy.dot, [ub, numpy.diag(sb), vhb])
)

cor_idx_b = []
act_idx_b = []
ext_idx_b = []

for p, sp in enumerate(sb):
    if abs(sp - 2.0) < tol:
        cor_idx_b.append(p)
    elif abs(sp) < tol:
        ext_idx_b.append(p)
    else:
        act_idx_b.append(p)

In [6]:
cas_nmo  = 0
cas_elec = 0.0

print(cor_idx_a, act_idx_a, ext_idx_a)
print(sum(sa[cor_idx_a]))
print(sum(sa[act_idx_a]))
print(sum(sa[ext_idx_a]))

cas_nmo  += len(sa[act_idx_a])
cas_elec += sum(sa[act_idx_a])

print(cor_idx_b, act_idx_b, ext_idx_b)
print(sum(sa[cor_idx_b]))
print(sum(sa[act_idx_b]))
print(sum(sa[ext_idx_b]))

cas_nmo  += len(sa[act_idx_b])
cas_elec += sum(sa[act_idx_b])
cas_elec  = int(numpy.round(cas_elec))


[0, 1] [2, 3, 4, 5] [6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23]
3.995770725446117
3.9999999999999987
0.004229274553879073
[0, 1] [2, 3, 4, 5] [6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23]
3.995770725446117
3.9999999999999987
0.004229274553879073


In [7]:
print(cas_elec)
print(cas_nmo)

8
8


In [8]:
c_ao_lo_a = c_ao_lo[:,idx_a]
c_pio_a   = reduce(numpy.dot, [c_ao_lo_a, ua])
npio = c_ao_lo_a.shape[1]

# for pp in range(npio):
#     tools.cubegen.orbital(mol, f'./cube/ch3_a_{pp}_{sa[pp]:4.2e}.cube', c_pio_a[:,pp])

c_ao_lo_b = c_ao_lo[:,idx_b]
c_pio_b   = reduce(numpy.dot, [c_ao_lo_b, ub])
npio = c_ao_lo_b.shape[1]

# for pp in range(npio):
#     tools.cubegen.orbital(mol, f'./cube/ch3_b_{pp}_{sb[pp]:4.2e}.cube', c_pio_b[:,pp])

In [9]:
assert numpy.linalg.norm(
    reduce(numpy.dot, [c_pio_a.T, ovlp_ao, c_pio_a]) - numpy.eye(c_pio_a.shape[1])
)

In [10]:
nao, nmo = mf.mo_coeff.shape
coeff = numpy.empty([nao, nmo])

mo_occ = [0 for p in range(nmo)]

cor_list = []
cas_list = []
ext_list = []

count = 0

for p in cor_idx_a:
    pp = count
    mo_occ[pp] = 2 
    cor_list.append(pp)
    coeff[:, pp] = c_pio_a[:, p]
    count += 1

for p in cor_idx_b:
    pp = count
    mo_occ[pp] = 2
    cor_list.append(pp)
    coeff[:, pp] = c_pio_b[:, p]
    count += 1

for p in act_idx_a:
    pp = count
    cas_list.append(pp)
    coeff[:, pp] = c_pio_a[:, p]
    count += 1

for p in act_idx_b:
    pp = count
    cas_list.append(pp)
    coeff[:, pp] = c_pio_b[:, p]
    count += 1
    
for p in ext_idx_a:
    pp = count
    ext_list.append(pp)
    coeff[:, pp] = c_pio_a[:, p]
    count += 1

for p in ext_idx_b:
    pp = count
    ext_list.append(pp)
    coeff[:, pp] = c_pio_b[:, p]
    count += 1

assert count == nmo

In [11]:
cas_list

[4, 5, 6, 7, 8, 9, 10, 11]

In [12]:
assert numpy.linalg.norm(
    reduce(numpy.dot, [coeff.T, ovlp_ao, coeff]) - numpy.eye(nmo)
)


with open("./cube_list.log", "w") as f:
    for pp in cas_list:
        print(f"Processing {pp}")
        file_path = f'c2h6_{pp}'
        tools.cubegen.orbital(mol, "./cube/"+file_path+".cube", coeff[:,pp])
        f.write(file_path+"\n")

Processing 4
Processing 5
Processing 6
Processing 7
Processing 8
Processing 9
Processing 10
Processing 11


In [13]:
len(cas_list)

8

In [14]:
from pyscf import mcscf

mf.mo_coeff = coeff
mycas = mcscf.CASCI(mf, len(cas_list), len(cas_list))
mo = mycas.sort_mo(cas_list)
mycas.kernel()
mycas.verbose = 0
# mycas.analyze()

CASCI E = -78.0993559839132  E(CI) = -16.286127094696  S^2 = 0.0000000


In [15]:
2 * 0.441908424721**2

0.39056611167879146

In [16]:
2 * 0.513243626576**2

0.5268380404417691

In [27]:
ci = mycas.ci
if (ci is not None and
    (getattr(mycas.fcisolver, 'large_ci', None) or
     getattr(mycas.fcisolver, 'states_large_ci', None))):
    print('** Largest CI components **')
    print('  [alpha occ-orbitals] [beta occ-orbitals]            CI coefficient')
    res = mycas.fcisolver.large_ci(ci, mycas.ncas, mycas.nelecas,
                                    5e-2, return_strs=False)
    
    aa_list = []
    bb_list = []
    cc_list = []
    
    for c,ia,ib in res:
        aa = [a+len(cor_list) for a in ia]
        bb = [b+len(cor_list) for b in ib]
        # print('  %-20s %-30s %12.4e'%(aa, bb, c**2))
        cc = c**2
        aa_list.append(aa)
        bb_list.append(bb)
        cc_list.append(cc)
    
    idx_list = numpy.argsort(cc_list)[::-1][:10]
    
    cc_sum = 0.0
    for idx in idx_list:
        print('  %-20s %-30s %12.4e'%(aa_list[idx], aa_list[idx], cc_list[idx]))
        cc_sum += cc_list[idx]
    print("cc_sum = %6.4f"%cc_sum)

** Largest CI components **
  [alpha occ-orbitals] [beta occ-orbitals]            CI coefficient
  [4, 5, 6, 8]         [4, 5, 6, 8]                     9.4011e-02
  [4, 8, 9, 10]        [4, 8, 9, 10]                    9.3978e-02
  [4, 6, 8, 10]        [4, 6, 8, 10]                    5.4557e-02
  [4, 5, 8, 9]         [4, 5, 8, 9]                     5.4548e-02
  [4, 6, 8, 10]        [4, 6, 8, 10]                    5.3116e-02
  [4, 5, 6, 8]         [4, 5, 6, 8]                     5.3115e-02
  [4, 5, 8, 9]         [4, 5, 8, 9]                     5.3100e-02
  [4, 8, 9, 10]        [4, 8, 9, 10]                    5.3099e-02
  [4, 5, 6, 8]         [4, 5, 6, 8]                     4.0472e-02
  [4, 5, 8, 9]         [4, 5, 8, 9]                     4.0470e-02
cc_sum = 0.5905


In [18]:
cor_list

[0, 1, 2, 3]